In [225]:
import numpy as np
import torch, argparse, json
import torch.nn as nn # layer types
import torch.nn.functional as func # useful ML functions (activations) that are applied to layers
import torch.optim as optim
import torch.autograd as autograd

# Ex_A = np.loadtxt(fname="data/a.txt", dtype='int')
# Ex_B = np.loadtxt(fname="data/b.txt", dtype='int')
# Ex_C = np.loadtxt(fname="data/c.txt", dtype='int')

# print(Ex_A,"\n")
# print(Ex_B,"\n")
# print(Ex_C)

learning_rate = 0.01
epochs = 100
display_epochs = 10
test_size = 10000 #final of 10,000
train_size = 1000 # final of 1,000
seed = 492 #can be whatever, just remember it
nhidden = 100

In [226]:
np.random.seed(seed=seed)
#data_x = []

def create_dataset(test_size,train_size):
    in_len = 8
    out_len = 16
    size = test_size+train_size
    max_int = 2**(in_len-1)
    format_in = '{:0' + str(in_len) + 'b}'
    format_out = '{:0' + str(out_len) + 'b}'
    
    A = np.zeros((size, in_len))
    B = np.zeros((size, in_len))
    C = np.zeros((size, out_len))
    
    for i in range(size):
        int_a = np.random.randint(0, max_int)
        int_b = np.random.randint(0, max_int)
        
        A[i,:] = list(reversed([int(b) for b in format_in.format(int_a)]))
        B[i,:] = list(reversed([int(b) for b in format_in.format(int_b)]))
        C[i,:] = list([int(b) for b in format_out.format(int_a*int_b)])
    return A, B, C

data_a, data_b, data_c = create_dataset(test_size,train_size)
data_x = np.append(data_a,data_b, axis=1)
print(f'A tensor shape: {data_a.shape}')
print(f'B tensor shape: {data_b.shape}')
print(f'C tensor shape: {data_c.shape}')
print(f'X tensor shape: {data_x.shape}')

print('\n'+ str(data_a[1]))
print(data_b[1])
print(data_x[1:4])

x_train = data_x[:-train_size]
x_test = data_x[-train_size:]
c_train = data_c[:-train_size]
c_test = data_c[-train_size:]

TypeError: data type not understood

In [ ]:
class Multiplier(nn.Module):
    def __init__(self):
        super(Multiplier, self).__init__() 
        self.rnn= nn.RNN(16, nhidden)
        self.lin= nn.Linear(nhidden, 16) 
        
    def forward(self, x):
        rnnOut,_=self.rnn(x)
        h = func.relu(rnnOut)
        y = func.sigmoid(self.lin(h))
        return y
    
    def reset(self):
        self.rnn.reset_parameters()
        self.lin.reset_parameters()

In [ ]:
model = Multiplier().to(torch.device("cpu"))
loss = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# targets = torch.from_numpy(c_train)
# targets = targets.contiguous()
# print(targets.size()) 

In [ ]:
#train and test in one loop
obj_vals= []
obj_accs= []
cross_vals= []
cross_accs= []

model.reset() # reset your parameters

for epoch in range(epochs):
    #Training
    inputs = torch.from_numpy(x_train)
    targets = torch.from_numpy(c_train)
    #targets = torch.reshape(targets, (-1, 1))
    
    model.zero_grad()
    
    inputs = autograd.Variable(inputs.unsqueeze(1).float())
    seqLen = inputs.size(0)
    inputs = inputs.contiguous()
    inputs = model(inputs)
    
    targets = autograd.Variable(targets.unsqueeze(1).float())
    #targets = targets.type(torch.LongTensor)

    obj_val = loss(inputs, targets)
    obj_vals.append(obj_val.item())
    
    optimizer.zero_grad()
    obj_val.backward()
    optimizer.step()
    

    if (epoch+1) % display_epochs == 0:
        print ('Epoch [{}/{}]\t Training Loss: {:.4f}'.format(epoch+1, epochs, obj_val.item()))
    
    #Checking loss at each step to compare
    with torch.no_grad(): 
        # don't track calculations in the following scope for the purposes of gradients
        inputs = torch.from_numpy(x_test)
        targets = torch.from_numpy(c_test)
        #targets = torch.reshape(targets, (-1, 1))
        
        inputs = autograd.Variable(inputs.unsqueeze(1).float())
        seqLen = inputs.size(0)
        inputs = inputs.contiguous()
        inputs = model(inputs)
#         inputs = inputs.argmax(2)
#         print(inputs[epoch,0,0])
        targets = autograd.Variable(targets.unsqueeze(1).float())
        #targets = targets.type(torch.LongTensor)
#         targets = targets.argmax(2)
        correct = 0
        for i in range(16):
            if inputs[epoch,0,i]>0.5:
                 inputs[epoch,0,i] =1
            else:
                inputs[epoch,0,i] =0
                
            if targets[epoch,0,i]>0.5:
                 targets[epoch,0,i] =1
            else:
                targets[epoch,0,i] =0
                
            if inputs[epoch,0,i]==targets[epoch,0,i]:
                correct+=1
    
        cross_val = loss(inputs, targets)
        cross_vals.append(cross_val)
        cross_accs.append(100*correct/16)
        

    if (epoch+1) % display_epochs == 0:
        print ('Epoch [{}/{}]\t Test Loss: {:.4f}'.format(epoch+1, epochs, cross_val.item()))
        print ('Epoch [{}/{}]\t Test Accuracy: {:.4f}%'.format(epoch+1, epochs, np.average(cross_accs)))
        print ('Target:', targets[epoch])
        print ('Trained:', inputs[epoch],'\n')
        
print('Final training loss: {:.4f}'.format(obj_vals[-1]))
print('Final testing performance: \n Loss: {:.4f} \t Accuracy: {:0.2f}%'.format(cross_vals[-1],np.average(cross_accs)))